In [22]:
import konlpy
from konlpy.tag import Okt
import csv

In [23]:
okt = Okt

In [24]:
with open("F:/eunjin/NLP/data/newslist.csv", "r", encoding='utf-8') as nl:
    news = nl.readlines()

news = [nl.replace("\n", "") for nl in news]

In [19]:
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

---

In [30]:
import pandas as pd
import numpy as np

In [31]:
data = pd.read_csv("F:/eunjin/NLP/data/newslist.csv")

In [50]:
df = data.loc[:, ['title', 'url', 'text']]
df = df[df['text'].notnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1099
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1094 non-null   object
 1   url     1094 non-null   object
 2   text    1094 non-null   object
dtypes: object(3)
memory usage: 34.2+ KB


### csv로 저장

df.to_csv("./test.csv")